In [275]:
# import packages

import pandas as pd
import re

# Model Prediction Task I

You are tasked to build a predictive algorithm to determine the factors affecting prices of residential properties in Singapore. You need to provide insights to your reporting officer to detail one or more strategies in curbing housing prices inflation.

Your fellow colleagues should be able to access and contribute to your code to replicate the same insights. Their local devices do not have GPU access. Provide justification for any of the choices you have made.

### Exploratory Data Analysis

In [258]:
approval_1990_1999 = pd.read_csv("data/resale-flat-prices-based-on-approval-date-1990-1999.csv")
approval_2000_2012 = pd.read_csv("data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")
registration_2012_2014 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
registration_2015_2016 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv")
registration_2017 = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

In [259]:
print(f"resale-flat-prices-based-on-approval-date-1990-1999 with shape {approval_1990_1999.shape}\n", approval_1990_1999.head())
print(f"\nresale-flat-prices-based-on-approval-date-2000-feb-2012 {approval_2000_2012.shape}\n", approval_2000_2012.head())

resale-flat-prices-based-on-approval-date-1990-1999 with shape (287200, 10)
      month        town flat_type block       street_name storey_range  \
0  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
1  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     04 TO 06   
2  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
3  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     07 TO 09   
4  1990-01  ANG MO KIO    3 ROOM   216  ANG MO KIO AVE 1     04 TO 06   

   floor_area_sqm      flat_model  lease_commence_date  resale_price  
0            31.0        IMPROVED                 1977          9000  
1            31.0        IMPROVED                 1977          6000  
2            31.0        IMPROVED                 1977          8000  
3            31.0        IMPROVED                 1977          6000  
4            73.0  NEW GENERATION                 1976         47200  

resale-flat-prices-based-on-approval-date-2000-feb-2012 (

In [260]:
print(f"resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014 with shape {registration_2012_2014.shape}\n", registration_2012_2014.head())
print(f"\nresale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016 {registration_2015_2016.shape}\n", registration_2015_2016.head())
print(f"\nresale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv {registration_2017.shape}\n", registration_2017.head())

resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014 with shape (52203, 10)
      month        town flat_type block        street_name storey_range  \
0  2012-03  ANG MO KIO    2 ROOM   172   ANG MO KIO AVE 4     06 TO 10   
1  2012-03  ANG MO KIO    2 ROOM   510   ANG MO KIO AVE 8     01 TO 05   
2  2012-03  ANG MO KIO    3 ROOM   610   ANG MO KIO AVE 4     06 TO 10   
3  2012-03  ANG MO KIO    3 ROOM   474  ANG MO KIO AVE 10     01 TO 05   
4  2012-03  ANG MO KIO    3 ROOM   604   ANG MO KIO AVE 5     06 TO 10   

   floor_area_sqm      flat_model  lease_commence_date  resale_price  
0            45.0        Improved                 1986      250000.0  
1            44.0        Improved                 1980      265000.0  
2            68.0  New Generation                 1980      315000.0  
3            67.0  New Generation                 1984      320000.0  
4            67.0  New Generation                 1980      321000.0  

resale-flat-prices-based-on-regi

Upon reading the names of the 5 csv files, there are two distinct naming for datasets: approval and registration. However, given that approval ranges from 1990 to february 2012 and registration ranges from march 2012 and onwards, it seems more like a change in naming convention than completely different datasets given that the variables are mostly identical. The assumption is therefore that these 5 datasets are actually tracking the exact same thing, and thus can be combined into one full dataset.

However, there is some data manipulation required as the variable input data types and value styles differ across the years (i.e. datasets), with some datasets even having new variables. A deeper analysis of the differences between datasets to be combined will be conducted to determine the final dataset data types and standardisation of variables.

#### Create Combined Dataset 

In [255]:
"""
Input: <list<DataFrame>> list of dataframes
Description: returns each variable's datatype
Output: <DataFrame> dataframe comparing variables and datatypes
"""


def get_datatypes(dfs, df_names) :
    
    df_datatypes = pd.DataFrame()
    for i in range(len(dfs)) :
        df = dfs[i]
        df_name = df_names[i]
        col_datatypes = {}
        for col in df.columns :
            col_datatypes[col] = [type(df[col][0])]
        col_df = pd.DataFrame.from_dict(col_datatypes, orient='index').reset_index()
        col_df.columns = ["variable", f"data type_{df_name}"]
        if i == 0 :
            df_datatypes = col_df
        else :
            df_datatypes = df_datatypes.merge(col_df, on='variable', how='outer')


    return df_datatypes


In [262]:
dfs = [approval_1990_1999, approval_2000_2012, registration_2012_2014, registration_2015_2016, registration_2017]
get_datatypes(dfs, ["1990_1999", "2000_2012", "2012_2014", "2015_2016", "2017"])

,variable,data type_1990_1999,data type_2000_2012,data type_2012_2014,data type_2015_2016,data type_2017
0,block,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,flat_model,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,flat_type,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,floor_area_sqm,<class 'numpy.float64'>,<class 'numpy.float64'>,<class 'numpy.float64'>,<class 'numpy.float64'>,<class 'numpy.float64'>
4,lease_commence_date,<class 'numpy.int64'>,<class 'numpy.int64'>,<class 'numpy.int64'>,<class 'numpy.int64'>,<class 'numpy.int64'>
5,month,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
6,remaining_lease,NaN,NaN,NaN,<class 'numpy.int64'>,<class 'str'>
7,resale_price,<class 'numpy.int64'>,<class 'numpy.float64'>,<class 'numpy.float64'>,<class 'numpy.float64'>,<class 'numpy.float64'>
8,storey_range,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
9,street_name,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


all 5 datasets have the same datatypes for block, flat_model, flat_type, floor_area_sqm, lease_commence_date, month, storey range, street_name and town. For resale_price, 1990_1999 is only one using int64 while from 2000, float is used. Assuming this datatype input change happened starting from 2000, resale_price in 1990_1999 will be changed to float as well for consistency. For remaining_lease, it seems like it was only created after 2015, explaining why this variable did not exist from 1990 to 2014. However, 2017 onwards uses string to include months while 2015_2016 uses int64, only taking the number of years. To capture the true value of remaining_lease, we will maintain the datatype int64 and convert values into months instead. For flat_model, 1990_1999 capitalises all variable inputs while the other datasets did not. To standardise, flat_model will all be capitalised for ease later when encoding categorical variables.

In [276]:
"""
Input: <str> remaining_lease value (e.g. 61 years 04 months)
Description: parses a string and converts years and months into only months
Output: <int> number of months
"""
def convert_to_total_months(lease_str):
    if not isinstance(lease_str, str):
        return 0
    
    # Extract digit for years and digit for months
    years_match = re.search(r'(\d+)\s*years?', lease_str)
    months_match = re.search(r'(\d+)\s*months?', lease_str)
    
    years = int(years_match.group(1)) if years_match else 0
    months = int(months_match.group(1)) if months_match else 0
    
    return (years * 12) + months

In [277]:
"""
Input: <DataFrame> dataframes to be combined
Description:
    - change approval_1990_1999 resale_price datatype to float
    - capitalise flat_model
    - convert remaining_lease to months
Output: <DataFrame> combined dataframe
"""

def create_df(approval_1990_1999, approval_2000_2012, registration_2012_2014, registration_2015_2016, registration_2017) :
    
    # change approval_1990_1999 resale_price datatype
    approval_1990_1999["resale_price"] = approval_1990_1999["resale_price"].astype(float)
    
    # convert remaining_lease to months
    registration_2015_2016["remaining_lease"] = registration_2015_2016["remaining_lease"]*12
    registration_2017['remaining_lease'] = registration_2017['remaining_lease'].apply(convert_to_total_months)

    # create df
    df = pd.concat([approval_1990_1999, approval_2000_2012, registration_2012_2014, registration_2015_2016, registration_2017])
    
    # capitalise flat_model
    df["flat_model"] = df["flat_model"].str.upper()
    
    return df

In [278]:
approval_df = create_df(approval_1990_1999, approval_2000_2012, registration_2012_2014, registration_2015_2016, registration_2017)
approval_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
80369,2020-09,YISHUN,5 ROOM,716,YISHUN ST 71,07 TO 09,131.0,IMPROVED,1987,440000.0,795.0
80370,2020-09,YISHUN,5 ROOM,760,YISHUN ST 72,07 TO 09,122.0,IMPROVED,1987,458000.0,786.0
80371,2020-09,YISHUN,5 ROOM,835,YISHUN ST 81,04 TO 06,122.0,IMPROVED,1987,490000.0,796.0
80372,2020-09,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,146.0,MAISONETTE,1987,558000.0,795.0


# Model Prediction Task II

Since we are building a predictive algorithm, we will transform the categorical variables (town, storey_range, flat_model) using one hot encoding since most algorithms produce better results with numerical variables.

# Link Analysis Task I

# Link Analysis Task II

# Bonus Question: Link Analysis Task III 